In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import scipy
import scanpy as sc
import scvi
import os
import sklearn.cluster as cluster
import scdrs

In [13]:
fca = sc.read_h5ad("/project2/gilad/jpopp/ebQTL/data/fca/counts.subsampled.lognorm.mca.scdrs_preprocessed.h5ad")

In [51]:
eb = fca[fca.obs['celltype'] != "Cardiomyocytes"]

In [11]:
differentiation_gs = scdrs.util.load_gs("/project2/gilad/jpopp/ebQTL/data/gene_sets/c5.go.bp.Hs.symbols.differentiation.gs")

In [9]:
g = list(go_bp_gs.keys())[0]
g

'GOBP_OVARIAN_FOLLICLE_DEVELOPMENT'

In [16]:
geneset = differentiation_gs[g][0]

In [17]:
result = scdrs.score_cell(fca, geneset, n_ctrl=100)

Computing control scores: 100%|██████████| 100/100 [00:24<00:00,  4.06it/s]


In [54]:
eb_result = scdrs.score_cell(fca, geneset, n_ctrl=100)
eb_result['celltype'] = eb.obs['celltype']

Computing control scores: 100%|██████████| 100/100 [00:24<00:00,  4.14it/s]


In [19]:
result['celltype'] = fca.obs['celltype']
result

,raw_score,norm_score,mc_pval,pval,nlog10_pval,zscore,celltype
exp4-human-100well.CGTACCTATGGCTCTCTAT,0.055543,2.376311,0.019802,0.029637,1.528159,1.886149,Adrenocortical cells
exp4-human-100well.AGTAGCCATGGTATCCGCC,0.028187,1.206138,0.069307,0.123032,0.909983,1.159964,Adrenocortical cells
exp4-human-101well.TTTGCTACTTTCGGCCTTAC,0.068773,2.625826,0.019802,0.021186,1.673946,2.029843,Adrenocortical cells
exp4-human-101well.TAACTAAGGTTAGAATTAAC,0.015259,0.162876,0.326733,0.352909,0.452337,0.377477,Adrenocortical cells
exp4-human-101well.CCATCAGGTGGTATCCGCC,0.016028,0.644194,0.217822,0.222566,0.652542,0.763557,Adrenocortical cells
...,...,...,...,...,...,...,...
exp2-human-568_.TACGGTAGTGTCCTAGAGG,0.029506,0.186823,0.336634,0.345278,0.461831,0.398100,Lymphoid cells
exp3-human-111well.TCGCGCATTTAGATCTACT,0.000000,-3.086563,1.000000,1.000000,-0.000000,-10.000000,Lymphoid cells
exp3-human-186well.AACGTAATCTATCCATGACT,0.006468,-0.493984,0.564356,0.595742,0.224942,-0.242341,Lymphoid cells
exp3-human-90well.ATATGCCATTTATATACTAA,0.000000,-3.086563,1.000000,1.000000,-0.000000,-10.000000,Lymphoid cells


In [27]:
fca_types = list(fca.obs.celltype.cat.categories)
fca_palette = sns.color_palette(n_colors=len(fca_types))

In [52]:
eb_types = list(eb.obs.celltype.cat.categories)
eb_palette = sns.color_palette([fca_palette[fca_types.index(t)] for t in eb_types])

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 15))
sns.violinplot(data=result, x="celltype", y="norm_score", palette=fca_palette, inner=None, ax=ax[0])
sns.violinplot(data=eb_result, x="celltype", y="norm_score", palette=eb_palette, inner=None, ax=ax[1])

ax[0].title.set_text(g + " (FCA)")
ax[0].tick_params(rotation=90)
ax[1].title.set_text(g + " (EB)")
ax[1].tick_params(rotation=90)

fig.tight_layout(pad=2.0)
plt.savefig("/project2/gilad/jpopp/ebQTL/temp/test2.png")